# Spark SQL

#### Creación del DataFrame para los ejemplos

In [0]:
df = spark.read.option("sep",";").option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/pdi_sales_small.csv")
df.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Units: integer (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Country: string (nullable = true)



#### Vistas temporales

In [0]:
# Se crea la vista temporal para luego hacer la consulta (ventas de Canada)
df.createOrReplaceTempView("ventas")
ventasCanada = spark.sql("select * from ventas where trim(Country)='Canada'")
ventasCanada.show(3)

+---------+---------+---------------+-----+-------+-------+
|ProductID|     Date|            Zip|Units|Revenue|Country|
+---------+---------+---------------+-----+-------+-------+
|      725|1/15/1999|H1B            |    1|  115.4|Canada |
|     2235|1/15/1999|H1B            |    2|  131.1|Canada |
|      713|1/15/1999|H1B            |    1|  160.1|Canada |
+---------+---------+---------------+-----+-------+-------+
only showing top 3 rows



#### Vistas globales

In [0]:
# Se crea la vista global y se hace una consulta
df.createOrReplaceGlobalTempView("ventasg")
ventasCanadaG = spark.sql("select * from global_temp.ventasg where trim(Country)='Canada'")
ventasCanadaG.show(3)

+---------+---------+---------------+-----+-------+-------+
|ProductID|     Date|            Zip|Units|Revenue|Country|
+---------+---------+---------------+-----+-------+-------+
|      725|1/15/1999|H1B            |    1|  115.4|Canada |
|     2235|1/15/1999|H1B            |    2|  131.1|Canada |
|      713|1/15/1999|H1B            |    1|  160.1|Canada |
+---------+---------+---------------+-----+-------+-------+
only showing top 3 rows



In [0]:
# Sobre una nueva sesión de Spark, se hace la misma consulta
sparkNew = spark.newSession()
sparkNew.sql("select * from global_temp.ventasg where trim(Country)='Canada' limit 5").show()

+---------+---------+---------------+-----+-------+-------+
|ProductID|     Date|            Zip|Units|Revenue|Country|
+---------+---------+---------------+-----+-------+-------+
|      725|1/15/1999|H1B            |    1|  115.4|Canada |
|     2235|1/15/1999|H1B            |    2|  131.1|Canada |
|      713|1/15/1999|H1B            |    1|  160.1|Canada |
|      574| 6/5/2002|H1B            |    1|  869.1|Canada |
|       94|2/15/1999|H1B            |    1|  866.2|Canada |
+---------+---------+---------------+-----+-------+-------+



#### Tablas

In [0]:
# Crear una base de datos
spark.sql("create database if not exists bigdata")
spark.sql("use database bigdata")

Out[9]: DataFrame[]

In [0]:
# Listado de las bases de datos existentes
spark.catalog.listDatabases()
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     bigdata|
|     default|
+------------+



In [0]:
# Crear una tabla gestionada a partir del dataframe df
df.write.format("parquet").mode("overwrite").saveAsTable("ventast")

In [0]:
# Vemos la descripción de la tabla con sentencias sql
spark.sql("describe table ventast").show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|ProductID|      int|   null|
|     Date|   string|   null|
|      Zip|   string|   null|
|    Units|      int|   null|
|  Revenue|   double|   null|
|  Country|   string|   null|
+---------+---------+-------+



In [0]:
# O una consulta simple sobre la tabla
spark.sql("select distinct Country from ventast").show()

+-------+
|Country|
+-------+
|Germany|
|France |
|Canada |
|Mexico |
| France|
+-------+



In [0]:
# Creación de una tabla externa indicando la ruta del fichero donde se alojan los datos de la tabla
spark.sql("""CREATE TABLE ventase(ProductID INT, Date STRING, 
  Zip STRING, Units INT, Revenue DOUBLE, Country STRING) 
  USING csv OPTIONS (PATH 
  '/FileStore/tables/pdi_sales_small.csv')""")

Out[16]: DataFrame[]

In [0]:
# Crear la tabla externa usando la opción path
df.write.option("path", "/user/spark/ventas").saveAsTable("ventaset")

In [0]:
from pyspark.sql.functions import sum
df.groupBy("Country").count().show()

+-------+-----+
|Country|count|
+-------+-----+
|Germany|30059|
|France |30059|
|Canada |30060|
|Mexico |30060|
| France|    1|
+-------+-----+



In [0]:
df.groupBy("Country").agg(sum("Revenue")).show()

+-------+--------------------+
|Country|        sum(Revenue)|
+-------+--------------------+
|Germany|1.4982119999999512E7|
|France |1.2086961900000831E7|
|Canada |1.1642614200001905E7|
|Mexico | 1.139459870000116E7|
| France|               980.2|
+-------+--------------------+



### Usando código SQL

En el menú de la izquierda de Databricks, en **Data**, seleccionamos el fichero con el que queremos trabajar como tabla, 
y pinchamos en el botón inferior _Create table in notebook_.

En el notebook que se abre, hay que cambiar las opciones:

~~~
# CSV options
infer_schema = "false"   # cambiar a true
first_row_is_header = "false"  # cambiar a true
delimiter = ","    # en la tabla pdi_sales_small.csv es ";"
~~~

Ejecutar las celdas y ya tenemos la tabla _pdi_sales_small_csv_ creada

In [0]:
spark.sql("use database default")

Out[21]: DataFrame[]

In [0]:
%sql
select sum(Revenue), Country from pdi_sales_small_csv group by Country

sum(Revenue),Country
1.4982119999999512E7,Germany
1.2086961900000831E7,France
1.1642614200001905E7,Canada
1.139459870000116E7,Mexico
980.2,France


In [0]:
%sql
select Country, avg(Revenue) as ventas
from pdi_sales_small_csv
group by Country
order by ventas desc

Country,ventas
France,980.2
Germany,498.4237665923521
France,402.1079177617629
Canada,387.31251497012323
Mexico,379.0618330007039


In [0]:
%sql
select Country, sum(Units) as pedidos
from pdi_sales_small_csv
group by Country
order by pedidos desc

Country,pedidos
Germany,31746
France,31738
Canada,31148
Mexico,31095
France,1
